<h1> Load data from MongoDB </h1>

In [ ]:
from pymongo import MongoClient
import pandas as pd

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
client.list_database_names()

In [ ]:
db = client['Tech']
db.list_collection_names()

In [ ]:
customers = pd.DataFrame(db.Customers.find({"Contract": {"$exists": True},"customerID": {"$exists": True},"Dependents": {"$exists": True},"gender": {"$exists": True},"MonthlyCharges": {"$exists": True},"PaperlessBilling": {"$exists": True} ,"Partner": {"$exists": True},"SeniorCitizen": {"$exists": True},"Services": {"$exists": True},"tenure": {"$exists": True},"TotalCharges": {"$exists": True} },{"_id":0}))
print(customers.shape)
customers.head(5)

In [ ]:
from pandas import json_normalize
df2 = json_normalize(customers['Services'])
customers = customers.join(df2)
customers = customers.drop('Services', axis=1)
print(customers.shape)
customers.head(5)

In [ ]:
customers['PaperlessBilling'] = customers['PaperlessBilling'].map(lambda x: x[0])
customers
original_customers = customers
print(customers.shape)

In [ ]:
def create_dummies(data_frame, column_name):
    return pd.get_dummies(data_frame, columns=[column_name], prefix=column_name, drop_first=True,dtype=int)

def prep(df_features):
    # basic preperations of the data
    df_features = df_features.rename(columns=str.lower)
    
    # הפכיה לערכים מספריים על מנת להציג את הנתונים
    df_features['totalcharges'] = df_features['totalcharges'].str.strip() 
    df_features['totalcharges'] = pd.to_numeric(df_features['totalcharges'], errors='coerce')
   
    # Replace the n/a with an estimation of the total charge
    df_features['totalcharges'] = df_features['totalcharges'].fillna((df_features['monthlycharges']*df_features['tenure']))  

    df_features['charge_diff'] = df_features['totalcharges'] - (df_features['monthlycharges']*df_features['tenure'])
    # if the number is negative we assume the price raised during the tenure
    # if the number is 0 there was no change in price during the tenure
    # if the number is positive we assume the price dropped during the tenure
    df_features['price_remain'] = (df_features['charge_diff'] == 0).astype(int)
    df_features['price_raised'] = (df_features['charge_diff'] < 0).astype(int)
    df_features['price_dropped'] = (df_features['charge_diff'] > 0).astype(int)
    
    df_features.loc[df_features.gender == 'Male','gender']=0
    df_features.loc[df_features.gender == 'Female','gender']=1
    df_features['gender'] = df_features['gender'].astype(int)
    
    df_features.loc[df_features.partner == 'No','partner']=0
    df_features.loc[df_features.partner == 'Yes','partner']=1
    df_features['partner'] = df_features['partner'].astype(int)
    
    df_features.loc[df_features.dependents == 'No','dependents']=0
    df_features.loc[df_features.dependents == 'Yes','dependents']=1
    df_features['dependents'] = df_features['dependents'].astype(int)
    
    df_features.loc[df_features.phoneservice == 'No','phoneservice'] =0
    df_features.loc[df_features.phoneservice == 'Yes','phoneservice'] =1
    df_features['phoneservice'] = df_features['phoneservice'].astype(int)
    
    df_features.loc[df_features.paperlessbilling == 'No','paperlessbilling'] = 0
    df_features.loc[df_features.paperlessbilling == 'Yes','paperlessbilling'] =1
    df_features['paperlessbilling'] = df_features['paperlessbilling'].astype(int)
    
    df_features = create_dummies(df_features, 'contract')
    df_features = create_dummies(df_features, 'multiplelines')
    df_features = create_dummies(df_features, 'internetservice')
    df_features = create_dummies(df_features, 'onlinesecurity')
    df_features = create_dummies(df_features, 'onlinebackup')
    df_features = create_dummies(df_features, 'deviceprotection')
    df_features = create_dummies(df_features, 'techsupport')
    df_features = create_dummies(df_features, 'streamingtv')
    df_features = create_dummies(df_features, 'streamingmovies')
    df_features = create_dummies(df_features, 'paymentmethod')

    df_features['combined'] = df_features['internetservice_Fiber optic'] | df_features['internetservice_No'] | \
           df_features['onlinesecurity_No internet service'] | df_features['onlinesecurity_Yes'] | \
           df_features['onlinebackup_No internet service'] | df_features['onlinebackup_Yes'] | \
           df_features['deviceprotection_No internet service'] | df_features['deviceprotection_Yes'] | \
           df_features['techsupport_No internet service'] | df_features['techsupport_Yes'] | \
           df_features['streamingtv_No internet service'] | df_features['streamingtv_Yes'] | \
           df_features['streamingmovies_No internet service'] | df_features['streamingmovies_Yes']
    
    sum_all_combined = df_features['internetservice_Fiber optic'] + df_features['internetservice_No'] + \
           df_features['onlinesecurity_No internet service'] + df_features['onlinesecurity_Yes'] + \
           df_features['onlinebackup_No internet service'] + df_features['onlinebackup_Yes'] + \
           df_features['deviceprotection_No internet service'] + df_features['deviceprotection_Yes'] + \
           df_features['techsupport_No internet service'] + df_features['techsupport_Yes'] + \
           df_features['streamingtv_No internet service'] + df_features['streamingtv_Yes'] + \
           df_features['streamingmovies_No internet service'] + df_features['streamingmovies_Yes']
    df_features['combined_majority'] = (sum_all_combined > 6).astype(int)
    
    df_features.drop(['internetservice_Fiber optic', 'internetservice_No','onlinesecurity_No internet service','onlinesecurity_Yes'], axis=1, inplace=True)
    df_features.drop(['onlinebackup_No internet service','onlinebackup_Yes','deviceprotection_No internet service', 'deviceprotection_Yes'], axis=1, inplace=True)
    df_features.drop(['techsupport_No internet service', 'techsupport_Yes','streamingtv_No internet service', 'streamingtv_Yes'], axis=1, inplace=True)
    df_features.drop(['streamingmovies_No internet service', 'streamingmovies_Yes'], axis=1, inplace=True)

    X = df_features.drop(columns=['customerid'])
    
    # Let's convert all data to float because some modules warn against other types
    X = X.astype(float)

    X = X.reindex(sorted(X.columns), axis=1)

    return X

In [ ]:
X = prep(customers)
print(X.shape)

In [ ]:
import joblib

# Load the pipeline using joblib
filename = "customer_final_pipeline.pkl"
pipeline = joblib.load(filename)

print(f"Pipeline loaded from {filename}")

In [ ]:
# Predictions
y_pred = pipeline.predict(X)

In [ ]:
df_cvs = pd.DataFrame()
df_cvs['customerID'] = original_customers['customerID']
df_cvs['gender'] = original_customers['gender']
df_cvs['SeniorCitizen'] = original_customers['SeniorCitizen']
df_cvs['Partner'] = original_customers['Partner']
df_cvs['Dependents'] = original_customers['Dependents']
df_cvs['tenure'] = original_customers['tenure']
df_cvs['PhoneService'] = original_customers['PhoneService']
df_cvs['MultipleLines'] = original_customers['MultipleLines']
df_cvs['InternetService'] = original_customers['InternetService']
df_cvs['OnlineSecurity'] = original_customers['OnlineSecurity']
df_cvs['OnlineBackup'] = original_customers['OnlineBackup']
df_cvs['DeviceProtection'] = original_customers['DeviceProtection']
df_cvs['TechSupport'] = original_customers['TechSupport']
df_cvs['StreamingTV'] = original_customers['StreamingTV']
df_cvs['StreamingMovies'] = original_customers['StreamingMovies']
df_cvs['Contract'] = original_customers['Contract']
df_cvs['PaperlessBilling'] = original_customers['PaperlessBilling']
df_cvs['PaymentMethod'] = original_customers['PaymentMethod']
df_cvs['MonthlyCharges'] = original_customers['MonthlyCharges']
df_cvs['TotalCharges'] = original_customers['TotalCharges']
df_cvs['Churn'] = y_pred.astype(object)

In [ ]:
df_cvs.info()

In [ ]:
df_cvs.to_csv('inferred_churn.csv', index=False)